In [3]:
import streamlit as st
import folium
from streamlit_folium import st_folium
from geopy.geocoders import Nominatim
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import plotly.express as px
from folium import GeoJson, GeoJsonTooltip
import xlrd
pd.set_option('display.max_rows', None)
import requests
from math import radians, sin, cos, sqrt, atan2
from IPython.display import display

In [4]:
geolocator = Nominatim(user_agent="Navigator")

def geocode_address(address):
    geolocator = Nominatim(user_agent="Navigator")
    return geolocator.geocode(address)

In [6]:
address = "Skaldevägen 60"
POI_radius=500
location = geocode_address(address)
lat, lon = location.latitude, location.longitude
lat,lon

(59.3285363, 17.9360367)

In [7]:
overpass_url = "http://overpass-api.de/api/interpreter"
query = f"""
[out:json][timeout:25];
node(around:10000,{lat},{lon})["shop"="supermarket"];
out 1;
"""
response = requests.get(overpass_url, params={'data': query})
data = response.json()

if not data.get("elements"):
    print("⚠️ No supermarkets found nearby.")
else:
    el = data["elements"][0]
    name = el["tags"].get("name", "Unnamed Supermarket")
    s_lat, s_lon = el["lat"], el["lon"]
    print(f"🛒 Closest supermarket: {name}")
    print(f"📍 Coordinates: ({s_lat:.6f}, {s_lon:.6f})")

    # --- 3. Compute distance (optional, quick calc) ---
    R = 6371e3  # Earth radius in meters
    φ1, φ2 = radians(lat), radians(s_lat)
    Δφ = radians(s_lat - lat)
    Δλ = radians(s_lon - lon)
    a = sin(Δφ/2)**2 + cos(φ1)*cos(φ2)*sin(Δλ/2)**2
    distance = R * 2 * atan2(sqrt(a), sqrt(1 - a))
    print(f"📏 Distance: {distance:.0f} meters")

 

🛒 Closest supermarket: ICA nära Karlaplan
📍 Coordinates: (59.337217, 18.091122)
📏 Distance: 8848 meters


In [8]:
# --- 4. Plot on Folium map ---
m = folium.Map(location=[lat, lon], zoom_start=14)
folium.Marker([lat, lon], tooltip="Your Address", icon=folium.Icon(color="blue")).add_to(m)
folium.Marker([s_lat, s_lon], tooltip=f"{name} ({distance:.0f} m)",
              icon=folium.Icon(color="green")).add_to(m)
folium.PolyLine([(lat, lon), (s_lat, s_lon)], color="gray", dash_array="5,5").add_to(m)

display(m)